NVDA 股價大跌 5%，我看好他會在 1 個月內反彈回來，有什麼選擇權策略適合？


In [1]:
import numpy as np
from scipy.stats import norm


def black_scholes_merton(S, K, T, r, sigma, option_type="call"):
    """
    根據BSM模型計算歐式選擇權的理論價格。

    參數:
    S : float - 股票的當前價格
    K : float - 行使價格
    T : float - 到期時間（以年為單位）
    r : float - 無風險利率（以小數表示，如0.01表示1%）
    sigma : float - 股票價格的波動率（以小數表示，如0.2表示20%）
    option_type : str - 選擇權類型 ('call' 或 'put')

    返回:
    float - 選擇權的理論價格
    """
    # 檢查選擇權類型是否有效
    if option_type not in ["call", "put"]:
        raise ValueError("Option type must be 'call' or 'put'")

    # 計算d1和d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # 根據選擇權類型計算價格
    if option_type == "call":
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return option_price


# 假設參數
S = 250  # 當前NVDA股票價格
K = 260  # 行使價格
T = 1 / 12  # 到期時間，假設為1個月
r = 0.01  # 無風險利率（1%）
sigma = 0.2  # 波動率（20%）
option_type = "call"  # 看漲期權

# 計算選擇權價格
option_price = black_scholes_merton(S, K, T, r, sigma, option_type)
print(f"The theoretical price of the {option_type} option is: ${option_price:.2f}")

The theoretical price of the call option is: $2.23


In [36]:
""" 假設 Spot price (S) 從 950 跌至 800 附近,然後又快速回彈至 950 """

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

stock_prices = [950, 870, 830, 800, 840, 880, 840, 900, 940, 950]
days = list(range(1, len(stock_prices) + 1))

stock_data = pd.DataFrame({"Day": days, "Stock Price": stock_prices})

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 設定行使價格和到期時間
K_values = [850, 900, 940, 950, 960, 1000]
T_values = [1 / 24, 1 / 12, 2 / 12, 3 / 12]

# 計算選擇權價格
option_prices = {}
for K in K_values:
    for T in T_values:
        option_prices[(K, T)] = [
            black_scholes_merton(S, K, T, r, sigma) for S in stock_prices
        ]

# 創建子圖
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)

# 股票價格隨時間變化的線圖
fig.add_trace(
    go.Scatter(x=days, y=stock_prices, mode="lines", name="Stock Price"), row=1, col=1
)

# 固定到期時間,不同行使價格的選擇權價格線圖
fixed_T = T_values[1] * 12
for K in K_values:
    fig.add_trace(
        go.Scatter(
            x=days, y=option_prices[(K, T_values[1])], mode="lines", name=f"K={K}"
        ),
        row=2,
        col=1,
    )

# 固定行使價格,不同到期時間的選擇權價格線圖
fixed_K = K_values[1]
for T in T_values:
    fig.add_trace(
        go.Scatter(
            x=days,
            y=option_prices[(K_values[1], T)],
            mode="lines",
            name=f"T={T*12:.0f} months",
        ),
        row=3,
        col=1,
    )

# 更新圖表佈局
fig.update_layout(
    title="Stock and Call Option Prices Over Time",
    xaxis=dict(title="Time (Days)"),
    yaxis=dict(title="Stock Price"),
    yaxis2=dict(title=f"Call Price (T={fixed_T:.0f} month)"),
    yaxis3=dict(title=f"Call Price (K={fixed_K})"),
    width=800,
    height=800,
)

fig.show()

In [31]:
import numpy as np
import pandas as pd
from scipy.stats import norm


def bsm_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call


def evaluate_calls(
    stock_prices, days, target_price, r, sigma, strike_range, expiry_range
):
    S = stock_prices[-1]
    T_values = [t / 365 for t in expiry_range]

    results = []
    for K in strike_range:
        for T in T_values:
            call_price = bsm_call(S, K, T, r, sigma)
            target_call_price = bsm_call(target_price, K, T - days[-1] / 365, r, sigma)

            leverage = (target_price / K - 1) / (call_price / S - 1)
            delta = norm.cdf(
                (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
            )
            expected_return = target_call_price / call_price - 1

            results.append(
                [
                    K,
                    T * 365,
                    call_price,
                    target_call_price,
                    leverage,
                    delta,
                    expected_return,
                ]
            )

    columns = [
        "Strike",
        "Expiry",
        "Call_Price",
        "Target_Call_Price",
        "Leverage",
        "Delta",
        "Expected_Return",
    ]
    df = pd.DataFrame(results, columns=columns)

    df = df[(df["Delta"] >= 0.4) & (df["Delta"] <= 0.7)]
    df = df.sort_values(by=["Leverage", "Expected_Return"], ascending=False)
    df["Expected_Return"] = df["Expected_Return"].apply(lambda x: f"{x:.2%}")

    return df


# 模擬參數
stock_prices = [950, 870, 830, 800, 840, 880, 840, 900, 940, 950]
days = list(range(0, len(stock_prices)))
target_price = 1045  # 預期價格漲10%
r = 0.01  # 無風險利率
sigma = 0.3  # 波動率

strike_range = [800, 850, 900, 950, 1000]
expiry_range = [30, 60, 90]  # 到期日範圍(天)

best_strategies = evaluate_calls(
    stock_prices, days, target_price, r, sigma, strike_range, expiry_range
)
print("Best long call strategies:")
# print(best_strategies.head(5))
best_strategies

Best long call strategies:


,Strike,Expiry,Call_Price,Target_Call_Price,Leverage,Delta,Expected_Return
14,1000,90.0,37.120818,84.193236,-0.046830,0.400024,126.81%
9,950,30.0,32.964668,98.589223,-0.103595,0.520959,199.08%
10,950,60.0,46.816360,108.322997,-0.105183,0.529627,131.38%
11,950,90.0,57.514429,117.512671,-0.106444,0.536268,104.32%
7,900,60.0,75.174574,150.749260,-0.174956,0.698065,100.53%
8,900,90.0,84.859288,156.891813,-0.176914,0.675078,84.88%


In [32]:
import numpy as np
import pandas as pd
from scipy.stats import norm


def bsm_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call


def evaluate_calls_by_day(stock_prices, days, r, sigma, strike_range, expiry_range):
    results = []
    for day, S in zip(days, stock_prices):
        for K in strike_range:
            for T in expiry_range:
                T_years = T / 365
                call_price = bsm_call(S, K, T_years - day / 365, r, sigma)
                results.append([day, K, T, call_price])

    columns = ["Day", "Strike", "Expiry", "Call_Price"]
    df = pd.DataFrame(results, columns=columns)

    df_pivot = df.pivot_table(
        index=["Strike", "Expiry"], columns="Day", values="Call_Price"
    )
    df_pivot.columns.name = None
    df_pivot.index.names = ["Strike", "Expiry"]

    return df_pivot


# 模擬參數
stock_prices = [950, 870, 830, 800, 840, 880, 840, 900, 940, 950]
days = list(range(0, len(stock_prices)))
r = 0.01  # 無風險利率
sigma = 0.3  # 波動率

strike_range = [800, 850, 900, 950, 1000]
expiry_range = [30, 60, 90, 120, 240, 360]  # 到期日範圍(天)

call_prices_by_day = evaluate_calls_by_day(
    stock_prices, days, r, sigma, strike_range, expiry_range
)
print("Call prices by day:")
call_prices_by_day

Call prices by day:


0           1           2          3           4  \
Strike Expiry                                                              
800    30      151.280259   76.492254   45.018169  26.320949   51.307335   
       60      154.973918   85.429030   56.446764  38.412412   62.466012   
       90      159.689694   93.329452   65.438940  47.606042   71.378713   
       120     164.645435  100.390225   73.109930  55.346564   79.021249   
       240     183.649099  123.511155   97.164959  79.332202  103.092881   
       360     200.492162  141.993779  115.858445  97.829655  121.850089   
850    30      104.192737   40.493912   19.214424   9.065249   22.537021   
       60      111.801696   52.884617   31.382252  19.358741   35.362758   
       90      119.206638   62.512197   40.800117  27.861130   45.092098   
       120     126.099728   70.686346   48.784245  35.250122   53.276633   
       240     149.571006   96.210007   73.683017  58.820860   78.619345   
       360     168.823100  115.990844   92.966813  77.349146   98.154835   
900    30       63.328206   17.448345    6.384161   2.349798    7.611499   
       60       75.174574   29.763161   15.667113   8.683361   17.941245   
       90       84.859288   39.456476   23.836376  15.218500   26.673913   
       120      93.251308   47.729282   31.118745  21.415826   34.325726   
       240     119.957703   73.690058   54.908459  42.842013   58.911457   
       360     140.914474   93.880286   73.904046  60.584428   78.326092   
950    30       32.964668    6.031135    1.649791   0.463317    1.963954   
       60       46.816360   15.220690    7.046911   3.488946    8.175673   
       90       57.514429   23.500748   13.090697   7.794634   14.815549   
       120      66.576502   30.958725   19.028621  12.455957   21.187918   
       240      94.814589   55.570031   40.272996  30.707572   43.441405   
       360     116.671570   75.356019   58.263530  47.062403   61.979569   
1000   30       14.431985    1.675460    0.336039   0.071022    0.391523   
       60       26.939837    7.097201    2.874676   1.267086    3.366442   
       90       37.120818   13.247955    6.788457   3.764422    7.759151   
       120      45.928854   19.338110   11.191670   6.963514   12.569931   
       240      73.926326   41.315588   29.119627  21.698203   31.573252   
       360      95.888087   60.039918   45.597379  36.295319   48.680771   

                        5           6           7           8           9  
Strike Expiry                                                              
800    30       84.072059   50.434446  102.087214  140.795896  150.632641  
       60       92.294898   61.808491  108.588707  144.451434  153.696418  
       90       99.932844   70.830639  115.390930  149.432590  158.229717  
       120     106.861711   78.541225  121.826203  154.668129  163.153750  
       240     129.814668  102.746316  143.931294  174.448342  182.300360  
       360     148.283382  121.564435  162.117951  191.710041  199.296506  
850    30       45.069251   21.470005   58.912689   93.062220  102.162549  
       60       57.694576   34.632972   70.591184  101.332261  109.499771  
       90       67.464259   44.500657   80.037265  109.308620  117.037586  
       120      75.741598   52.765483   88.162469  116.610536  124.084784  
       240     101.530856   78.257210  113.808611  140.943296  147.987069  
       360     121.486110   97.858083  133.814084  160.578512  167.481429  
900    30       19.233544    6.857308   27.308558   51.987387   59.099270  
       60       32.637557   17.308936   41.649632   65.367829   71.908775  
       90       42.838880   26.129935   52.291437   75.763552   82.116302  
       120      51.430515   33.841984   61.172306   84.575995   90.839957  
       240      78.061978   58.554273   88.475723  112.034565  118.207250  
       360      98.605651   78.029420  109.425227  133.278724  139.469113  
950    30        6.371444    1.629402    9.781764   23.292710   27.5324

In [34]:
import numpy as np
import pandas as pd
from scipy.stats import norm


def bsm_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call


def evaluate_call_performance(stock_prices, days, r, sigma, strike_range, expiry_range):
    results = []
    for K in strike_range:
        for T in expiry_range:
            T_years = T / 365
            call_prices = [
                bsm_call(S, K, T_years - day / 365, r, sigma)
                for day, S in zip(days, stock_prices)
            ]
            returns = np.diff(call_prices) / call_prices[:-1]

            total_return = (call_prices[-1] - call_prices[0]) / call_prices[0]
            volatility = np.std(returns, ddof=1) * np.sqrt(252)
            downside_risk = np.sqrt(np.mean([min(0, r) ** 2 for r in returns]) * 252)
            sharpe_ratio = np.mean(returns) / np.std(returns, ddof=1) * np.sqrt(252)

            cumulative_returns = (1 + returns).cumprod() - 1
            max_drawdown = np.max(
                np.maximum.accumulate(cumulative_returns) - cumulative_returns
            )
            max_drawup = np.max(
                cumulative_returns - np.minimum.accumulate(cumulative_returns)
            )

            results.append(
                [
                    K,
                    T,
                    total_return,
                    volatility,
                    downside_risk,
                    sharpe_ratio,
                    max_drawdown,
                    max_drawup,
                ]
            )

    columns = [
        "Strike",
        "Expiry",
        "Total_Return",
        "Volatility",
        "Downside_Risk",
        "Sharpe_Ratio",
        "Max_Drawdown",
        "Max_Drawup",
    ]
    df = pd.DataFrame(results, columns=columns)

    df["Total_Return"] = df["Total_Return"].apply(lambda x: f"{x:.2%}")
    df["Volatility"] = df["Volatility"].apply(lambda x: f"{x:.2%}")
    df["Downside_Risk"] = df["Downside_Risk"].apply(lambda x: f"{x:.2%}")
    df["Max_Drawdown"] = df["Max_Drawdown"].apply(lambda x: f"{x:.2%}")
    df["Max_Drawup"] = df["Max_Drawup"].apply(lambda x: f"{x:.2%}")

    return df


# 模擬參數
stock_prices = [950, 870, 830, 800, 840, 880, 840, 900, 940, 950]
days = list(range(0, len(stock_prices)))
r = 0.01  # 無風險利率
sigma = 0.3  # 波動率

strike_range = [800, 850, 900, 950, 1000]
expiry_range = [30, 60, 90]  # 到期日範圍(天)

call_performance = evaluate_call_performance(
    stock_prices, days, r, sigma, strike_range, expiry_range
)
print("Call option performance:")
# print(call_performance)
call_performance

Call option performance:


,Strike,Expiry,Total_Return,Volatility,Downside_Risk,Sharpe_Ratio,Max_Drawdown,Max_Drawup
0,800,30,-0.43%,980.72%,457.12%,3.825270,33.16%,82.17%
1,800,60,-0.82%,744.50%,384.37%,3.072726,30.34%,74.39%
2,800,90,-0.91%,634.03%,343.36%,2.668562,28.63%,69.27%
3,850,30,-1.95%,1482.47%,580.40%,5.134730,30.16%,89.35%
4,850,60,-2.06%,958.95%,458.13%,3.784567,29.99%,80.63%
5,850,90,-1.82%,770.45%,397.13%,3.158729,29.07%,74.81%
6,900,30,-6.68%,2235.38%,698.07%,6.457100,23.84%,89.61%
7,900,60,-4.34%,1216.46%,531.39%,4.529803,28.04%,84.10%
8,900,90,-3.23%,923.57%,450.81%,3.668893,28.56%,78.83%
9,950,30,-16.48%,3352.56%,796.62%,7.552309,16.89%,82.12%


In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm


def bsm_call(S, K, T, r, sigma):
    """
    Black-Scholes-Merton model to calculate European call option price.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call


def calculate_call_prices(stock_prices, K, T_years, r, sigma):
    """
    Calculate call prices for a given set of parameters.

    Parameters:
    - stock_prices: list of stock prices
    - K: strike price
    - T_years: time to expiry in years
    - r: risk-free interest rate
    - sigma: volatility of the stock

    Returns:
    - DataFrame containing the call prices.
    """
    days = np.arange(len(stock_prices))
    call_prices = [
        bsm_call(S, K, T_years - day / 365, r, sigma)
        for day, S in zip(days, stock_prices)
    ]
    return pd.DataFrame({"day": days, "call_price": call_prices})


def calculate_metrics(call_prices_df):
    """
    Calculate various performance metrics based on call prices.

    Parameters:
    - call_prices_df: DataFrame with call prices.

    Returns:
    - A dictionary containing various performance metrics.
    """
    call_prices = call_prices_df["call_price"]
    returns = np.diff(call_prices) / call_prices[:-1]

    total_return = (call_prices.iloc[-1] - call_prices.iloc[0]) / call_prices.iloc[0]
    volatility = np.std(returns, ddof=1) * np.sqrt(252)
    downside_risk = np.sqrt(np.mean([min(0, r) ** 2 for r in returns]) * 252)
    sharpe_ratio = np.mean(returns) / np.std(returns, ddof=1) * np.sqrt(252)

    cumulative_returns = np.cumprod(1 + returns) - 1
    max_drawdown = np.max(
        np.maximum.accumulate(cumulative_returns) - cumulative_returns
    )
    max_drawup = np.max(cumulative_returns - np.minimum.accumulate(cumulative_returns))

    return {
        "total_return": total_return,
        "volatility": volatility,
        "downside_risk": downside_risk,
        "sharpe_ratio": sharpe_ratio,
        "max_drawdown": max_drawdown,
        "max_drawup": max_drawup,
    }


# Simulation parameters
# stock_prices = [950, 870, 830, 800, 840, 880, 840, 900, 940, 950]
stock_prices = [900, 870, 830, 800, 840, 880, 840, 900, 940, 950]
days = list(range(0, len(stock_prices)))
r = 0.01  # Risk-free interest rate
sigma = 0.3  # Volatility

strike_range = [700, 800, 850, 900, 950, 1000, 1100]
expiry_range = [30, 60, 90, 120, 240, 360]  # Expiry dates range in days

# Initialize a list to store the results
results = []

# Iterate through each combination of strike price and expiry
for K in strike_range:
    for T in expiry_range:
        T_years = T / 365
        call_prices_df = calculate_call_prices(stock_prices, K, T_years, r, sigma)
        metrics = calculate_metrics(call_prices_df)
        results.append([K, T] + list(metrics.values()))


# Create a DataFrame to display the results
results_df = pd.DataFrame(
    results,
    columns=[
        "Strike Price",
        "Expiry (Days)",
        "Total Return",
        "Volatility",
        "Downside Risk",
        "Sharpe Ratio",
        "Max Drawdown",
        "Max Drawup",
    ],
)


# Display the results table
# print(results_df)
results_df

,Strike Price,Expiry (Days),Total Return,Volatility,Downside Risk,Sharpe Ratio,Max Drawdown,Max Drawup
0,700,30,0.248218,4.329427,2.212450,3.272017,0.343300,0.740812
1,700,60,0.244067,4.084324,2.101418,3.239908,0.325606,0.715307
2,700,90,0.236884,3.839964,1.989725,3.196431,0.309599,0.687040
3,700,120,0.229038,3.630589,1.892951,3.153007,0.296247,0.660772
4,700,240,0.202430,3.061623,1.624010,3.016589,0.259669,0.581488
5,700,360,0.183967,2.722393,1.459134,2.926601,0.236924,0.529448
6,800,30,0.456000,9.393523,3.994348,4.690422,0.484951,1.201585
7,800,60,0.394751,7.015671,3.248110,4.154762,0.426662,1.046170
8,800,90,0.353203,5.914285,2.845196,3.872606,0.391033,0.946069
9,800,120,0.323571,5.238343,2.579072,3.689802,0.365413,0.874577


In [11]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Assuming results_df is already populated with the data from the previous step

# Initialize subplot
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=("Total Return", "Volatility", "Sharpe Ratio", "Max Drawdown"),
)


# Custom function to create hover text
def create_hover_text(row):
    return f"Strike: {row['Strike Price']}, Expiry: {row['Expiry (Days)']} days"


# Add traces with hoverinfo
metrics = ["Total Return", "Volatility", "Sharpe Ratio", "Max Drawdown"]
positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for metric, pos in zip(metrics, positions):
    fig.add_trace(
        go.Scatter(
            x=results_df["Strike Price"],
            y=results_df[metric],
            mode="markers",
            name=metric,
            text=results_df.apply(create_hover_text, axis=1),
            hoverinfo="text+y",
        ),
        row=pos[0],
        col=pos[1],
    )

# Update layout for readability
fig.update_layout(
    height=800,
    width=1200,
    title_text="Option Performance Metrics by Strike Price with Strike and Expiry Details",
    hovermode="closest",
)

fig.show()